In [142]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.legend_handler import HandlerLine2D
# import json
from pylab import figure
import ast
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from dtw import dtw
import functools
import math
%pylab inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

from keras.models import Sequential
from keras.layers import LSTM
from keras.utils import to_categorical

import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
def load_dataframe(filepath):
    data = pd.read_csv(filepath, sep=", ", header=None)
    data[0] = data[0].apply(lambda x: x.replace("[(", ""))
    data[2] = data[2].apply(lambda x: x.replace(")", ""))
    data[3] = data[3].apply(lambda x: x.replace("(", ""))
    data[5] = data[5].apply(lambda x: x.replace(")]", ""))
    for i in range(len(data.columns)):
        data[i] = data[i].apply(lambda x: int(x))
    data.columns = ['accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
    return data 

In [3]:
def create_single_row_df(df):
    df1=df.stack().swaplevel()
    df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
    df_new = df1.to_frame().T
    c = df_new.shape[1] + 1
    while c <= 2880:
        df_new[c] = np.nan
        c += 1     
    return df_new

In [4]:
# edit this as per the file name 
labels = {'BT': 1, 'LT': 2, 'RT': 3, 'FT': 4, 'BFS': 5, 'LRS': 6, 'RLS': 7, 'FBS': 8}

In [5]:
def gesture_overall_dataframe(root_dir, label):
    final_df = pd.DataFrame()
    label_df = pd.DataFrame()
    for fname in os.listdir(root_dir):
        if fname == '.DS_Store':
            continue
        filepath = os.path.join(root_dir, fname)
        df = create_single_row_df(load_dataframe(filepath))
        df['label'] = label
        #dataframes.append(df)
        final_df = final_df.append(pd.DataFrame(df.values.tolist()), ignore_index=True)
    #dataframe = pd.concat([each for each in dataframes],ignore_index=True)    
    return final_df

In [6]:
def get_dataframe_for_each_participant(participant_number):
    dfs = []
    for l in labels:
        gesture_name = participant_number + "_" + l
        # Temporary adjustment 
#         if gesture_name == 'P2_Left':
#             continue
        dfs.append(gesture_overall_dataframe(gesture_name, labels[l]))
    df =  pd.concat([each for each in dfs],ignore_index=True)      
    return df

In [7]:
def get_dataframe_for_all_participants():
    particiapnts = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12']
    dfs = []
    for p in particiapnts:
        dfs.append(get_dataframe_for_each_participant(p))
    df = pd.concat([each for each in dfs],ignore_index=True)      
    return df        

In [ ]:
# Get the same row table from the original dataframe 
# Loop from 0 to 221 and then for each row --> have one function to write one row in 6 files, and add the label in another file. And check after every 5 

In [94]:
# def dataframe_to_folders(df_all):
#     acc_x = []
#     acc_y = []
#     acc_z = []
#     gyro_x = []
#     gyro_y = []
#     gyro_z = []
#     labels = [] 
#     for i in range(len(df_all)):
#         df = pd.DataFrame(df_all.iloc[[i],:-1].values) # getting all values excep the labels 
#         dfc = pd.DataFrame(df.values.reshape(380,-1),
#                       index=df.index.repeat(380)) # this is like the original load_dataframe style like dataframe  
#         labels.append(int(df_all[[df_all.shape[1] - 1]].loc[[i]].values[0][0])) # append the labels   
        
#         # pad with 0 and convert columns to rows
#         acc_x.append(dfc[[0]].fillna(0).T)
#         acc_y.append(dfc[[1]].fillna(0).T)
#         acc_z.append(dfc[[2]].fillna(0).T)
#         gyro_x.append(dfc[[3]].fillna(0).T)
#         gyro_y.append(dfc[[4]].fillna(0).T)
#         gyro_z.append(dfc[[5]].fillna(0).T)
#     acc_x_df = pd.concat([each for each in acc_x],ignore_index=True)       
#     acc_y_df = pd.concat([each for each in acc_y],ignore_index=True)
#     acc_z_df = pd.concat([each for each in acc_z],ignore_index=True)
#     gyro_x_df = pd.concat([each for each in gyro_x],ignore_index=True)
#     gyro_y_df = pd.concat([each for each in gyro_y],ignore_index=True)
#     gyro_z_df = pd.concat([each for each in gyro_z],ignore_index=True)
#     labels_df = pd.DataFrame(labels)
#     return acc_x_df, acc_y_df, acc_z_df, gyro_x_df, gyro_y_df, gyro_z_df, labels_df

In [8]:
sample_dataframe = get_dataframe_for_all_participants()
sample_dataframe

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0,-13367.0,-1662.0,-10121.0,-38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-13362.0,-1695.0,-10161.0,-55.0,426.0,-7.0,-13383.0,-1700.0,-10162.0,-49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-13144.0,-1888.0,-10396.0,-75.0,424.0,2.0,-13136.0,-1877.0,-10397.0,-77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-13108.0,-1612.0,-10541.0,-130.0,325.0,1.0,-13108.0,-1612.0,-10541.0,-130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-13148.0,-2318.0,-10280.0,-38.0,378.0,12.0,-13144.0,-2319.0,-10308.0,-57.0,...,-127.0,381.0,23.0,-13331.0,-2095.0,-10071.0,-127.0,381.0,23.0,1.0
6,-13366.0,-1200.0,-10235.0,15.0,428.0,-17.0,-13366.0,-1200.0,-10235.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-13430.0,-1341.0,-10081.0,15.0,470.0,-28.0,-13404.0,-1354.0,-10107.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-13382.0,-1596.0,-10150.0,-114.0,334.0,-1.0,-13384.0,-1608.0,-10135.0,-136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-13390.0,-1665.0,-10154.0,-32.0,423.0,-19.0,-13396.0,-1666.0,-10176.0,-76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [9]:
sad = sample_dataframe.copy()
sad

,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0,-13367.0,-1662.0,-10121.0,-38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-13362.0,-1695.0,-10161.0,-55.0,426.0,-7.0,-13383.0,-1700.0,-10162.0,-49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-13144.0,-1888.0,-10396.0,-75.0,424.0,2.0,-13136.0,-1877.0,-10397.0,-77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-13108.0,-1612.0,-10541.0,-130.0,325.0,1.0,-13108.0,-1612.0,-10541.0,-130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-13148.0,-2318.0,-10280.0,-38.0,378.0,12.0,-13144.0,-2319.0,-10308.0,-57.0,...,-127.0,381.0,23.0,-13331.0,-2095.0,-10071.0,-127.0,381.0,23.0,1.0
6,-13366.0,-1200.0,-10235.0,15.0,428.0,-17.0,-13366.0,-1200.0,-10235.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-13430.0,-1341.0,-10081.0,15.0,470.0,-28.0,-13404.0,-1354.0,-10107.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-13382.0,-1596.0,-10150.0,-114.0,334.0,-1.0,-13384.0,-1608.0,-10135.0,-136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-13390.0,-1665.0,-10154.0,-32.0,423.0,-19.0,-13396.0,-1666.0,-10176.0,-76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [116]:
sad = sad.fillna(0)
sad

,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0,-13367.0,-1662.0,-10121.0,-38.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-13362.0,-1695.0,-10161.0,-55.0,426.0,-7.0,-13383.0,-1700.0,-10162.0,-49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-13144.0,-1888.0,-10396.0,-75.0,424.0,2.0,-13136.0,-1877.0,-10397.0,-77.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-13108.0,-1612.0,-10541.0,-130.0,325.0,1.0,-13108.0,-1612.0,-10541.0,-130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,-13148.0,-2318.0,-10280.0,-38.0,378.0,12.0,-13144.0,-2319.0,-10308.0,-57.0,...,-127.0,381.0,23.0,-13331.0,-2095.0,-10071.0,-127.0,381.0,23.0,1.0
6,-13366.0,-1200.0,-10235.0,15.0,428.0,-17.0,-13366.0,-1200.0,-10235.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,-13430.0,-1341.0,-10081.0,15.0,470.0,-28.0,-13404.0,-1354.0,-10107.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,-13382.0,-1596.0,-10150.0,-114.0,334.0,-1.0,-13384.0,-1608.0,-10135.0,-136.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,-13390.0,-1665.0,-10154.0,-32.0,423.0,-19.0,-13396.0,-1666.0,-10176.0,-76.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [106]:
ddd = pd.DataFrame(sad.iloc[[0],:-1].values) ## Get each row 
ddd_c = pd.DataFrame(ddd.values.reshape(480,-1),
                      index=ddd.index.repeat(480))
ddd_c

,0,1,2,3,4,5
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13429.0,-1614.0,-10105.0,-29.0,376.0,-38.0
0,-13429.0,-1614.0,-10105.0,-29.0,376.0,-38.0


In [107]:
bbb = pd.DataFrame(sad.iloc[[1],:-1].values) ## Get each row 
bbb_c = pd.DataFrame(bbb.values.reshape(480,-1),
                      index=bbb.index.repeat(480))
bbb_c

,0,1,2,3,4,5
0,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0
0,-13367.0,-1662.0,-10121.0,-38.0,357.0,-17.0
0,-13367.0,-1662.0,-10121.0,-38.0,357.0,-17.0
0,-13367.0,-1662.0,-10121.0,-38.0,357.0,-17.0
0,-13367.0,-1662.0,-10121.0,-38.0,357.0,-17.0
0,-13367.0,-1662.0,-10121.0,-38.0,357.0,-17.0
0,-13367.0,-1662.0,-10121.0,-38.0,357.0,-17.0
0,-13320.0,-1686.0,-10152.0,-66.0,359.0,-20.0
0,-13320.0,-1686.0,-10152.0,-66.0,359.0,-20.0
0,-13320.0,-1686.0,-10152.0,-66.0,359.0,-20.0


### Randomize the row orderings here for pd dataframe 'sad'

In [130]:
X_train = []
labels = []
for i in range(len(sad)):
    df = pd.DataFrame(sad.iloc[[i],:-1].values) # getting all values excep the labels 
    dfc = pd.DataFrame(df.values.reshape(480,-1),
                  index=df.index.repeat(480)) # this is like the original load_dataframe style like dataframe  
    labels.append([(int(sad[[sad.shape[1] - 1]].loc[[i]].values[0][0]))])
    X_train.append(dfc.values)        

In [136]:
X_train = numpy.array(X_train)
X_train.shape

(222, 380, 6)

In [137]:
y_train = numpy.array(labels)
y_train.shape

(222, 1)

In [146]:
def one_hot(y_, n_classes=4):
    # Function to encode neural one-hot output labels from number indexes
    # e.g.:
    # one_hot(y_=[[5], [0], [3]], n_classes=6):
    #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]

    y_ = y_.reshape(len(y_))
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

In [151]:
to_categorical([[5], [0], [3]], 6).argmax(1)

array([5, 0, 3])

In [147]:
## Remember to normalize your data

In [141]:
# define model where LSTM is also output layer
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(222,6))) # Add a mask layer over here for zeroes 
model.add(LSTM(32)
model.add(Dense(10, activation='softmax'))